In [ ]:
import pandas as pd
import pylab as plt
%matplotlib inline

In [ ]:
def add_multirow_xticks(ax, dmsp_flux_fh):
    """Add multirow tickmarks to the bottom axis as is common in the
    magnetospheres community.

    Args
      ax: matplotlib axes
      dmsp_flux_fh: file handle (as returned by read_dmsp_flux_file)
    """
    xticks = ax.get_xticks()
    new_labels = []
    
    for time_float in xticks:
        time = num2date(time_float)
        i = dmsp_flux_fh['t'].searchsorted(time)
        mlat = dmsp_flux_fh['mlat'][i]
        mlt = dmsp_flux_fh['mlt'][i]
        
        new_label = '%s\n%.1f\n%.1f' % (
            time.strftime('%H:%M:%S'), mlat, mlt
        )
        new_labels.append(new_label)
        
    ax.set_xticklabels(new_labels)
    ax.text(-0.075, -0.075, 'Time', transform=ax.transAxes)
    ax.text(-0.075, -0.15, 'MLAT', transform=ax.transAxes)
    ax.text(-0.075, -0.225, 'MLT', transform=ax.transAxes)
    

def adjust_axis_energy_yticks(ax):
    """Adjust yticklabels for axes with y-axis being energy.
    
    Sets them to terms like eV and keV.

    Args
      ax: matplotlib axes    
    """
    yticks = 10**ax.get_yticks()
    labels = []

    for ytick in yticks:
        if ytick < 1000:
            labels.append('%d eV' % ytick)
        else:
            labels.append('%.1f keV' % (ytick/1000))

    ax.set_yticklabels(labels)

In [ ]:
df_human = pd.read_csv('data/Dec20_2015_storm_human_selecteted.csv', parse_dates=['start_time', 'end_time'])
df_human.sort_values(['sat', 'start_time'], inplace=True)
print(f'{len(df_human.index)} human selections')

df_human.head()


In [ ]:
import lib_search_dispersion
import glob
import os
import warnings
from matplotlib import MatplotlibDeprecationWarning
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.dates import num2date
warnings.filterwarnings("ignore")

for _, row_match in df_human.iterrows():
    
    fname = glob.glob(f'data/**/{os.path.basename(row_match.filename)}', recursive=True)[0]
    dmsp_flux_fh = lib_search_dispersion.read_dmsp_flux_file(fname)

    orig_i = dmsp_flux_fh['t'].searchsorted(row_match.start_time)
    orig_j = dmsp_flux_fh['t'].searchsorted(row_match.end_time)

    delta_index = int(0.50 * (orig_j - orig_i))  # make plot 50% wider
    i = max(orig_i - delta_index, 0)
    j = min(orig_j + delta_index, dmsp_flux_fh['t'].size - 1)
    fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

    # Plot title
    time_length = row_match.end_time - row_match.start_time
    title = (
        f'DMSP Plasma Spectrograms F{row_match.sat} ('
        f'{row_match.start_time.isoformat()} to ' +
        f'{row_match.end_time.isoformat()})'
    )

    axes[0].set_title(title)

    # Ion spectrogram
    im = axes[0].pcolor(
        dmsp_flux_fh['t'][i:j],
        np.log10(dmsp_flux_fh['ch_energy']),
        dmsp_flux_fh['ion_d_ener'][:, i:j], 
        norm=LogNorm(vmin=1e3, vmax=1e8), cmap='jet'
    )
    plt.colorbar(im, ax=axes[0]).set_label('Energy Flux')

    #axes[0].plot(dmsp_flux_fh['t'][orig_i:orig_j], Eic[orig_i:orig_j],
    #             'b*-')
    axes[0].axhline(
        np.log10(lib_search_dispersion.MAX_EIC_ENERGY),
        color='black', linestyle='dashed'
    )
    axes[0].set_ylabel('Ions\nLog Energy [eV]')
    adjust_axis_energy_yticks(axes[0])

    # Electron Spectrogram
    im = axes[1].pcolor(
        dmsp_flux_fh['t'][i:j],
        np.log10(dmsp_flux_fh['ch_energy']),
        dmsp_flux_fh['el_d_ener'][:, i:j], 
        norm=LogNorm(vmin=1e5, vmax=1e10), cmap='jet'
    )
    plt.colorbar(im, ax=axes[1]).set_label('Energy Flux')
    axes[1].set_ylabel('Electrons\nLog Energy [eV]')
    adjust_axis_energy_yticks(axes[1])

    add_multirow_xticks(axes[-1], dmsp_flux_fh)

    # Plot spacings
    plt.subplots_adjust(hspace=.05)
    plt.tight_layout()

    # Save image
    #out_name = outfolder + '/'
    #out_name += f'{os.path.basename(dmsp_flux_file)}_'
    #out_name += f"{row_match.start_time.isoformat()}_"
    #out_name += f"{row_match.end_time.isoformat()}.png"

    #os.makedirs(outfolder, exist_ok=True)
    #plt.savefig(out_name)
    #plt.close()

    #cprint('Wrote plot ' + out_name, 'green')

    #df_match['file'] = dmsp_flux_file